<a href="https://colab.research.google.com/github/adityakulkarni0910/Heartbeat_classification/blob/main/Heartbeat_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from datetime import datetime
from pathlib import Path
import pandas as pd

import torchtext.data as ttd

In [ ]:
folder = Path('/content/drive/My Drive/Study/DL/Assignment & HW/HW5')

#Task 1

##Pre-processing

In [ ]:
data = pd.read_csv(folder / 'hearbeat_classification_data.csv', encoding='ISO-8859-1')

In [ ]:
data.describe

<bound method NDFrame.describe of       att1@NUMERIC  att2@NUMERIC  ...  att140@NUMERIC  target
0         1.469756     -1.048520  ...       -1.039932       1
1        -1.998602     -3.770552  ...        0.799517       1
2        -1.187772     -3.365038  ...       -0.824489       1
3         0.604969     -1.671363  ...        0.023843       1
4        -1.197203     -3.270123  ...        1.734676       1
...            ...           ...  ...             ...     ...
4995     -0.248881     -1.346474  ...       -0.378163       0
4996     -0.287286     -1.199089  ...        0.075742       0
4997     -1.032096     -2.811901  ...        0.750669       1
4998     -1.592541     -2.461370  ...        0.354016       0
4999     -1.945586     -3.840519  ...       -1.366044       1

[5000 rows x 141 columns]>

In [ ]:
data['target'].value_counts()

1    2919
0    2081
Name: target, dtype: int64

In [ ]:
corpus = (data.drop('target',axis=1)).values
labels = data['target'].values

In [ ]:
# separate into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    corpus,  # predictors
    labels,  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

In [ ]:
X_train = torch.tensor(X_train).float()
y_train=torch.tensor(y_train)

In [ ]:
X_test = torch.tensor(X_test).float()
y_test=torch.tensor(y_test)

In [ ]:
from torch.utils import data
train_dataset = data.TensorDataset(X_train,y_train)

In [ ]:
test_dataset = data.TensorDataset(X_test,y_test)

In [ ]:
batch_size=16
train_loader= torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=4)

In [ ]:
test_loader= torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False,
                                        num_workers=4)

In [ ]:
# Check out what the data loader does
# makes the data of shape (batch size, color, height, width)
for inputs, targets in train_loader:
  print("input shape:", inputs.shape, "output shape:", targets.shape)
  break

input shape: torch.Size([16, 140]) output shape: torch.Size([16])


##1.1 Simple RNN

In [ ]:
### Define simple RNN
class SimpleRNN(nn.Module):
  def __init__(self, n_inputs, n_hidden, n_rnnlayers, n_outputs):
    super(SimpleRNN, self).__init__()
    self.D = n_inputs
    self.M = n_hidden
    self.K = n_outputs
    self.L = n_rnnlayers

    # note: batch_first=True
    # applies the convention that our data will be of shape:
    # (num_samples, sequence_length, num_features)
    # rather than:
    # (sequence_length, num_samples, num_features)
    self.rnn = nn.RNN(
        input_size=self.D,
        hidden_size=self.M,
        num_layers=self.L,
        nonlinearity='relu',
        batch_first=True)
    self.fc = nn.Linear(self.M, self.K)
  
  def forward(self, X):
    # initial hidden states
    h0 = torch.zeros(self.L, X.size(0), self.M).to(device)

    # get RNN unit output
    # out is of size (N, sequence_length, M)
    # 2nd return value is hidden states at each hidden layer
    # we don't need those now
    out, _ = self.rnn(X, h0)

    # we only want h(T) at the final time step
    # N x M -> N x K
    out = self.fc(out[:, -1, :])
    return out

In [ ]:
# upload model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
model_1 = SimpleRNN(n_inputs=1, n_hidden=10, n_rnnlayers=1, n_outputs=2)
model_1.to(device)

SimpleRNN(
  (rnn): RNN(1, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
#In the training loop below, I changed the input dimensions "input.view(-1,140,1)" 

In [ ]:
epochs =5
lr= 0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_1.parameters(), lr)

# TRAIN THE MODEL

train_losses= np.zeros(epochs)
test_losses= np.zeros(epochs)

for epoch in range(epochs):
  
  t0= datetime.now()
  train_loss=[]
  
  model_1.train()
  for input,targets in train_loader:

    input = input.to(device)
    targets= targets.to(device)
 
    input= (input.view(-1,140,1)) #

    # forward pass
    output= model_1(input)

    loss=criterion(output,targets)
    

    # set gradients to zero 
    optimizer.zero_grad()

    # backward pass
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
  
  train_loss=np.mean(train_loss)
      
  test_loss=[]
  model_1.eval()
  with torch.no_grad():
    for input,targets in test_loader:
      # load input and output to GPU
      input = (input.float()).to(device)
      targets= targets.to(device)
      
      # reshape the input
      input = input.view(-1,140,1) #
      # forward pass
      output= model_1(input)
      loss=criterion(output,targets)
      test_loss.append(loss.item())

    test_loss=np.mean(test_loss)
  
  # save Losses
  train_losses[epoch]= train_loss
  test_losses[epoch]= test_loss

  dt= datetime.now()-t0
  print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}    Test Loss: {test_loss:.4f}, Duration: {dt}')

Epoch 1/5, Train Loss: 0.5440    Test Loss: 0.3328, Duration: 0:00:04.145648
Epoch 2/5, Train Loss: 0.2526    Test Loss: 0.2057, Duration: 0:00:03.849462
Epoch 3/5, Train Loss: 0.2098    Test Loss: 0.1854, Duration: 0:00:04.321926
Epoch 4/5, Train Loss: 0.2005    Test Loss: 0.1788, Duration: 0:00:04.277346
Epoch 5/5, Train Loss: 0.1972    Test Loss: 0.1740, Duration: 0:00:03.997787


In [ ]:
# Accuracy- write a function to get accuracy
# use this function to get train/test accuracy and print accuracy
def get_accuracy(train_loader, test_loader, model):
  model.eval()
  with torch.no_grad():
    correct_train=correct_test=0
    total_train=total_test=0
    
    for input, targets in train_loader:
      input= input.to(device)
      targets= targets.to(device)
      input = input.view(-1,140,1) #
      output=model(input)
      _,indices = torch.max(output,dim=1)
      correct_train+= (targets==indices).sum().item()
      total_train += targets.shape[0]
    
    train_acc= correct_train/total_train

    for input, targets in test_loader:
      input= input.to(device)
      targets= targets.to(device)
      input = input.view(-1,140,1) #
      output=model(input)
      _,indices = torch.max(output,dim=1)
      correct_test+= (targets==indices).sum().item()
      total_test += targets.shape[0]
    
    test_acc= correct_test/total_test
    return train_acc, test_acc

In [ ]:
train_acc, test_acc = get_accuracy(train_loader, test_loader, model_1)
print(f'Train acc: {train_acc:.4f},\t Test acc: {test_acc:.4f}')

Train acc: 0.9357,	 Test acc: 0.9470


##1.2 LSTM

In [ ]:
### Define simple RNN
class LSTM(nn.Module):
  def __init__(self, n_inputs, n_hidden, n_rnnlayers, n_outputs):
    super(LSTM, self).__init__()
    self.D = n_inputs
    self.M = n_hidden
    self.K = n_outputs
    self.L = n_rnnlayers

    # note: batch_first=True
    # applies the convention that our data will be of shape:
    # (num_samples, sequence_length, num_features)
    # rather than:
    # (sequence_length, num_samples, num_features)
    self.rnn = nn.LSTM(
        input_size=self.D,
        hidden_size=self.M,
        num_layers=self.L,
 #       nonlinearity='relu',
        batch_first=True)
    self.fc = nn.Linear(self.M, self.K)
  
  def forward(self, X):
    # initial hidden states
    h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
    c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

    # get RNN unit output
    # out is of size (N, sequence_length, M)
    # 2nd return value is hidden states at each hidden layer
    # we don't need those now
    out, _ = self.rnn(X, (h0, c0))

    # we only want h(T) at the final time step
    # N x M -> N x K
    out = self.fc(out[:, -1, :])
    return out

In [ ]:
# upload model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
model_3 = LSTM(n_inputs=1, n_hidden=10, n_rnnlayers=1, n_outputs=2)
model_3.to(device)

LSTM(
  (rnn): LSTM(1, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=2, bias=True)
)

In [ ]:
#In the training loop below, I changed the input dimensions "input.view(-1,140,1)" 

In [ ]:
epochs =5
lr= 0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_3.parameters(), lr)

# TRAIN THE MODEL

train_losses= np.zeros(epochs)
test_losses= np.zeros(epochs)

for epoch in range(epochs):
  
  t0= datetime.now()
  train_loss=[]
  
  model_3.train()
  for input,targets in train_loader:

    input = input.to(device)
    targets= targets.to(device)
 
    input= (input.view(-1,140,1)) #

    # forward pass
    output= model_3(input)

    loss=criterion(output,targets)
    

    # set gradients to zero 
    optimizer.zero_grad()

    # backward pass
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
  
  train_loss=np.mean(train_loss)
      
  test_loss=[]
  model_3.eval()
  with torch.no_grad():
    for input,targets in test_loader:
      # load input and output to GPU
      input = (input.float()).to(device)
      targets= targets.to(device)
      
      # reshape the input
      input = input.view(-1,140,1) #
      # forward pass
      output= model_3(input)
      loss=criterion(output,targets)
      test_loss.append(loss.item())

    test_loss=np.mean(test_loss)
  
  # save Losses
  train_losses[epoch]= train_loss
  test_losses[epoch]= test_loss

  dt= datetime.now()-t0
  print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}    Test Loss: {test_loss:.4f}, Duration: {dt}')

Epoch 1/5, Train Loss: 0.6679    Test Loss: 0.6324, Duration: 0:00:09.654106
Epoch 2/5, Train Loss: 0.5961    Test Loss: 0.5483, Duration: 0:00:09.527208
Epoch 3/5, Train Loss: 0.4932    Test Loss: 0.4280, Duration: 0:00:09.707292
Epoch 4/5, Train Loss: 0.3692    Test Loss: 0.3115, Duration: 0:00:09.415029
Epoch 5/5, Train Loss: 0.2843    Test Loss: 0.2548, Duration: 0:00:09.625608


In [ ]:
# Accuracy- write a function to get accuracy
# use this function to get train/test accuracy and print accuracy
def get_accuracy(train_loader, test_loader, model):
  model.eval()
  with torch.no_grad():
    correct_train=correct_test=0
    total_train=total_test=0
    
    for input, targets in train_loader:
      input= input.to(device)
      targets= targets.to(device)
      input = input.view(-1,140,1) #
      output=model(input)
      _,indices = torch.max(output,dim=1)
      correct_train+= (targets==indices).sum().item()
      total_train += targets.shape[0]
    
    train_acc= correct_train/total_train

    for input, targets in test_loader:
      input= input.to(device)
      targets= targets.to(device)
      input = input.view(-1,140,1) #
      output=model(input)
      _,indices = torch.max(output,dim=1)
      correct_test+= (targets==indices).sum().item()
      total_test += targets.shape[0]
    
    test_acc= correct_test/total_test
    return train_acc, test_acc

In [ ]:
train_acc, test_acc = get_accuracy(train_loader, test_loader, model_3)
print(f'Train acc: {train_acc:.4f},\t Test acc: {test_acc:.4f}')

Train acc: 0.9265,	 Test acc: 0.9320
